# Make PDF files as Retrieval-Augmented Generation Embedding

## Data Preparation

### Packages

In [1]:
# required libraries

# !pip install pypdf PyPDF2 langchain transformers datasets 

In [2]:
import os
import PyPDF2

ModuleNotFoundError: No module named 'PyPDF2'

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

ModuleNotFoundError: No module named 'langchain'

### Load

### RAG

### Embed

### Save